In [1]:
import argparse
import os
import shutil
from src.analysis.project_parser import parse_project
from src.analysis.dependency_resolver import resolve_dependencies
from src.analysis.feature_mapper import map_features_to_components
from src.generation.project_generator import generate_project
from src.generation.feature_integration import integrate_features
from src.models.llm_inference import query_llm
from src.models.prompt_templates import get_prompt_template, get_prompt_template_feature
from src.utils.logger import logger
from src.utils.file_operations import read_file, write_file
from src.vector_database.db_query import query_vector_database

In [2]:
from src.vector_database.db_load import load_vector_database

In [3]:
logger.info("Starting the Factory Feature program")
# Define paths
old_project_path = "project_old"
new_project_path = "project_new"

In [4]:
# Step 1: Parse the old project
logger.info("Parsing the project structure")
project_data = parse_project(old_project_path)

In [5]:
project_data

[{'path': 'project_old/app.py',
  'content': 'from utils.helpers import greet\n\ndef main():\n    name = input("Enter your name: ")\n    print(greet(name))\n\nif __name__ == "__main__":\n    main()'},
 {'path': 'project_old/requirements.txt',
  'content': '# Python dependencies\nflask'},
 {'path': 'project_old/utils/helpers.py',
  'content': 'def greet(name):\n    """\n    Returns a greeting message for the provided name.\n\n    Args:\n        name: The name of the person to greet.\n\n    Returns:\n        A greeting string.\n    """\n    return f"Hello, {name}! Welcome to the project."'}]

In [6]:
project_data

[{'path': 'project_old/app.py',
  'content': 'from utils.helpers import greet\n\ndef main():\n    name = input("Enter your name: ")\n    print(greet(name))\n\nif __name__ == "__main__":\n    main()'},
 {'path': 'project_old/requirements.txt',
  'content': '# Python dependencies\nflask'},
 {'path': 'project_old/utils/helpers.py',
  'content': 'def greet(name):\n    """\n    Returns a greeting message for the provided name.\n\n    Args:\n        name: The name of the person to greet.\n\n    Returns:\n        A greeting string.\n    """\n    return f"Hello, {name}! Welcome to the project."'}]

In [7]:
type(project_data)

list

In [8]:
from src.vector_database.db_builder_simple import build_vector_database_simple


In [9]:
vector_db = build_vector_database_simple(project_data)
logger.info("Vector database simple created for the project")

/mnt/c/Blog/Factory-Feature/src/vector_database/db_builder_simple.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
vector_db

In [11]:
from src.vector_database.db_builder import build_vector_database


In [12]:
# With persistence
vector_db = build_vector_database(project_data, persist_directory="./chroma_db")

/mnt/c/Blog/Factory-Feature/src/vector_database/db_builder.py:35: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


In [13]:
vector_db = load_vector_database(persist_directory="./chroma_db")

/mnt/c/Blog/Factory-Feature/src/vector_database/db_load.py:63: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(


In [14]:
vector_db

In [15]:
from src.vector_database.display_content import display_first_five_documents
# Display the first 5 documents in the vector database
display_first_five_documents(vector_db)

Displaying the first 5 documents:
Document ID: 6434b73e-f406-4852-8ff9-a9a3d252c8fc
Content:
Path: project_old/app.py
Content:
from utils.helpers import greet

def main():
    name = input("Enter your name: ")
    print(greet(name))

if __name__ == "__main__":
    main()
Metadata: {'source': 'project_old/app.py'}
--------------------------------------------------------------------------------
Document ID: adeca2f7-04af-4ce6-b30f-2041cebb7270
Content:
Path: project_old/requirements.txt
Content:
# Python dependencies
flask
Metadata: {'source': 'project_old/requirements.txt'}
--------------------------------------------------------------------------------
Document ID: b70470ba-b70f-466e-aa3e-1ed31c73faf3
Content:
Path: project_old/utils/helpers.py
Content:
def greet(name):
    """
    Returns a greeting message for the provided name.

    Args:
        name: The name of the person to greet.

    Returns:
        A greeting string.
    """
    return f"Hello, {name}! Welcome to the project

In [16]:
# Step 2: Resolve dependencies
logger.info("Resolving project dependencies")
dependencies = resolve_dependencies(old_project_path)
logger.info(f"Dependencies found: {dependencies.keys()}")

In [17]:
# Step 3: Map features to project components
logger.info("Mapping feature request to project components")
feature_template = get_prompt_template_feature()
project_context = "\n".join([f"{key}: {value}" for key, value in dependencies.items()])


In [18]:
project_context

"requirements.txt: ['# Python dependencies\\n', 'flask']"

In [19]:
from src.analysis.tree import get_tree

if __name__ == "__main__":
    old_project_path = "project_old"
    try:
        # Get the tree structure without content
        tree = get_tree(old_project_path)
        print(tree)
        
        # Get the tree structure with content
        tree_with_content = get_tree(old_project_path, content=True)
        print(tree_with_content)
    except FileNotFoundError as e:
        print(e)


{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}
{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py', 'content': 'from utils.helpers import greet\n\ndef main():\n    name = input("Enter your name: ")\n    print(greet(name))\n\nif __name__ == "__main__":\n    main()'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt', 'content': '# Python dependencies\nflask'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py', 'content': 'def greet(name):\n    """\n    Returns a greeting message f

In [20]:
tree = get_tree(old_project_path)


In [21]:
tree

{'project_name': 'project_old',
 'structure': [{'type': 'file',
   'name': 'app.py',
   'path': 'project_old/app.py'},
  {'type': 'file',
   'name': 'requirements.txt',
   'path': 'project_old/requirements.txt'},
  {'type': 'directory',
   'name': 'utils',
   'path': 'project_old/utils',
   'children': [{'type': 'file',
     'name': 'helpers.py',
     'path': 'project_old/utils/helpers.py'}]}]}

In [22]:
tree_with_content = get_tree(old_project_path, content=True)


In [23]:
tree_with_content

{'project_name': 'project_old',
 'structure': [{'type': 'file',
   'name': 'app.py',
   'path': 'project_old/app.py',
   'content': 'from utils.helpers import greet\n\ndef main():\n    name = input("Enter your name: ")\n    print(greet(name))\n\nif __name__ == "__main__":\n    main()'},
  {'type': 'file',
   'name': 'requirements.txt',
   'path': 'project_old/requirements.txt',
   'content': '# Python dependencies\nflask'},
  {'type': 'directory',
   'name': 'utils',
   'path': 'project_old/utils',
   'children': [{'type': 'file',
     'name': 'helpers.py',
     'path': 'project_old/utils/helpers.py',
     'content': 'def greet(name):\n    """\n    Returns a greeting message for the provided name.\n\n    Args:\n        name: The name of the person to greet.\n\n    Returns:\n        A greeting string.\n    """\n    return f"Hello, {name}! Welcome to the project."'}]}]}

In [24]:
user_request="Add logging functionality to all major modules in the project"

In [25]:
tree

{'project_name': 'project_old',
 'structure': [{'type': 'file',
   'name': 'app.py',
   'path': 'project_old/app.py'},
  {'type': 'file',
   'name': 'requirements.txt',
   'path': 'project_old/requirements.txt'},
  {'type': 'directory',
   'name': 'utils',
   'path': 'project_old/utils',
   'children': [{'type': 'file',
     'name': 'helpers.py',
     'path': 'project_old/utils/helpers.py'}]}]}

In [26]:
tree_string = str(tree) 
print(tree_string)

{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}


In [27]:
### ATTENTTION WE ARE PARSING ONLY THE TREE WITHOUT THE CONTENT
### Maybe we can parse the full tree with content but we need to think a solution 
## Such the project_context does not supperate the limit of tokens.
## Maybe chunking or another technique.

project_context=project_context+tree_string

In [28]:
project_context

"requirements.txt: ['# Python dependencies\\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}"

In [29]:
#feature_request = feature_template.format(feature_request=user_request, project_context=project_context)


In [30]:
feature_request=user_request

In [31]:
# Step 0: Verification of LLM Connection
from src.models.llm_inference import query_llm
prompt = "Who is man's best friend?"
response = query_llm(prompt)
print(response)

Debug: Received prompt: Who is man's best friend?
Debug: Vector DB provided: False
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Who is man's best friend?<|eot_id|><|start_header_id|>assistant<|end_header_id|>



/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying without vector database...
Debug: Raw response from model: That's an easy one! Man's best friend is the dog! Dogs have been human companions for thousands of years, providing loyalty, affection, and protection. They are often referred to as "man's best friend" because of their unique ability to form strong bonds with humans and provide comfort, companionship, and assistance.
Debug: Response as string: That's an easy one! Man's best friend is the dog! Dogs have been human companions for thousands of years, providing loyalty, affection, and protection. They are often referred to as "man's best friend" because of their unique ability to form strong bonds with humans and provide comfort, companionship, and assistance.
That's an easy one! Man's best friend is the dog! Dogs have been human companions for thousands of years, providing loyalty, affection, and protection. They are often referred to as "man's best friend" because of their unique ability to form strong bonds with 

In [32]:
response

'That\'s an easy one! Man\'s best friend is the dog! Dogs have been human companions for thousands of years, providing loyalty, affection, and protection. They are often referred to as "man\'s best friend" because of their unique ability to form strong bonds with humans and provide comfort, companionship, and assistance.'

In [33]:
# Step 1: Feature Analysis
logger.info("Performing feature analysis")
feature_nodes_template = get_prompt_template("feature_analysis_nodes")
feature_edges_template = get_prompt_template("feature_analysis_edges")
impact_report_template = get_prompt_template("feature_impact_report")


## Feature Analysis without VectorDB

In [34]:
#1 feature_nodes

In [35]:
feature_nodes = feature_nodes_template.format(
    feature_request=user_request, project_context=project_context
)

In [36]:
feature_nodes_response = query_llm(user_input=feature_nodes)

Debug: Received prompt: Analyze the project context and identify the nodes (files or components) impacted by the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Project Context: requirements.txt: ['# Python dependencies\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}

Provide a list of impacted nodes and their roles in the project.
Debug: Vector DB provided: False
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Analyze the project context and identify the nodes (files or components) impacted by the following feature 

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying without vector database...
Debug: Raw response from model: Based on the project context and the feature request, the impacted nodes are:

1. **app.py**: This is the main application file, and it's likely that logging functionality will need to be added to this file to log events and errors related to the application's core functionality. **Role: Main Application File**

2. **helpers.py**: This file is part of the utils module, and it may contain helper functions that are used throughout the application. Adding logging functionality to this file will help track the usage and behavior of these helper functions. **Role: Utility Functions**

3. **requirements.txt**: Although this file doesn't need to be directly modified to add logging functionality, it may need to be updated to include a logging library (e.g., Python's built-in `logging` module or a third-party library like `loguru`) as a dependency. **Role: Dependency Management**

4. **New logging configuration file (e.g

In [37]:
#2 feature_edges

In [38]:
feature_edges = feature_edges_template.format(
    feature_request=user_request, project_context=project_context
)

In [39]:
feature_edges_response = query_llm(user_input=feature_edges)

Debug: Received prompt: Analyze the project context and determine the dependencies (edges) impacted by the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Project Context: requirements.txt: ['# Python dependencies\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}

Provide a list of impacted dependencies and their significance.
Debug: Vector DB provided: False
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Analyze the project context and determine the dependencies (edges) impacted by the following feature request:

Fea

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying without vector database...
Debug: Raw response from model: Based on the project context and the feature request, the impacted dependencies (edges) are:

1. **app.py -> logging module**: The feature request requires adding logging functionality to all major modules, including app.py. This means that app.py will now depend on the logging module, which is a Python standard library module.

Significance: High, as app.py is a major module in the project and logging functionality is a critical aspect of the feature request.

2. **helpers.py -> logging module**: Similar to app.py, helpers.py is also a major module in the project, and adding logging functionality to it will introduce a dependency on the logging module.

Significance: Medium to High, as helpers.py is a utility module that may be used by other parts of the project, and logging functionality is important for debugging and monitoring.

3. **requirements.txt -> logging module (indirectly)**: Although requirements.tx

In [40]:
#3 impact_report

In [41]:
impact_report = impact_report_template.format(
    feature_request=user_request, project_context=project_context
)

In [42]:
impact_report_response = query_llm(user_input=impact_report)

Debug: Received prompt: Generate a detailed report on the impact of the following feature request on the project:

Feature Request: Add logging functionality to all major modules in the project

Project Context: requirements.txt: ['# Python dependencies\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}

Include risks, suggestions for modular implementation, and expected changes.
Debug: Vector DB provided: False
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Generate a detailed report on the impact of the following feature request on the project:

Feature Request: Add logging f

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying without vector database...
Debug: Raw response from model: **Impact Report: Adding Logging Functionality to Major Modules**

**Introduction**

The proposed feature request aims to add logging functionality to all major modules in the project. This report assesses the impact of this request on the project, highlighting potential risks, suggestions for modular implementation, and expected changes.

**Current Project Structure**

The project, named `project_old`, has a simple structure consisting of:

1. `app.py`: The main application file.
2. `requirements.txt`: A file listing Python dependencies, currently only Flask.
3. `utils` directory: Containing a single file, `helpers.py`, with utility functions.

**Impact Analysis**

**Benefits**

1. **Improved Debugging**: Logging functionality will enable developers to track errors, warnings, and informational messages, making it easier to identify and debug issues.
2. **Enhanced Monitoring**: Logs will provide valuable insights

## Feature Analysis with VectorDB

In [43]:
vector_db = load_vector_database(persist_directory="./chroma_db")

In [44]:
from src.vector_database.db_load import load_vector_db_by_collection
from src.models.llm_inference import query_llm

In [45]:
feature_nodes = feature_nodes_template.format(
    feature_request=user_request, project_context=project_context
)


In [46]:
feature_nodes

"Analyze the project context and identify the nodes (files or components) impacted by the following feature request:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nProject Context: requirements.txt: ['# Python dependencies\\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}\n\nProvide a list of impacted nodes and their roles in the project."

In [47]:
feature_nodes_response_db = query_llm(user_input=feature_nodes,vector_db=vector_db)

Debug: Received prompt: Analyze the project context and identify the nodes (files or components) impacted by the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Project Context: requirements.txt: ['# Python dependencies\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}

Provide a list of impacted nodes and their roles in the project.
Debug: Vector DB provided: True
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Analyze the project context and identify the nodes (files or components) impacted by the following feature r

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying using vector database...
Debug: Raw response from vector DB: {'query': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nNone<|eot_id|><|start_header_id|>user<|end_header_id|>\nAnalyze the project context and identify the nodes (files or components) impacted by the following feature request:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nProject Context: requirements.txt: ['# Python dependencies\\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}\n\nProvide a list of impacted nodes and their roles in the project.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n", 'result': ' \n\nBased on the proj

In [48]:
feature_edges = feature_edges_template.format(
    feature_request=user_request, project_context=project_context
)

feature_edges_response_db = query_llm(user_input=feature_edges,vector_db=vector_db)

Debug: Received prompt: Analyze the project context and determine the dependencies (edges) impacted by the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Project Context: requirements.txt: ['# Python dependencies\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}

Provide a list of impacted dependencies and their significance.
Debug: Vector DB provided: True
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Analyze the project context and determine the dependencies (edges) impacted by the following feature request:

Feat

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying using vector database...
Debug: Raw response from vector DB: {'query': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nNone<|eot_id|><|start_header_id|>user<|end_header_id|>\nAnalyze the project context and determine the dependencies (edges) impacted by the following feature request:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nProject Context: requirements.txt: ['# Python dependencies\\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}\n\nProvide a list of impacted dependencies and their significance.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n", 'result': " \n\nBased on the project con

In [49]:
impact_report = impact_report_template.format(
    feature_request=user_request, project_context=project_context
)

impact_report_response_db = query_llm(user_input=impact_report,vector_db=vector_db)

Debug: Received prompt: Generate a detailed report on the impact of the following feature request on the project:

Feature Request: Add logging functionality to all major modules in the project

Project Context: requirements.txt: ['# Python dependencies\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}

Include risks, suggestions for modular implementation, and expected changes.
Debug: Vector DB provided: True
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Generate a detailed report on the impact of the following feature request on the project:

Feature Request: Add logging fu

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying using vector database...
Debug: Raw response from vector DB: {'query': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nNone<|eot_id|><|start_header_id|>user<|end_header_id|>\nGenerate a detailed report on the impact of the following feature request on the project:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nProject Context: requirements.txt: ['# Python dependencies\\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}\n\nInclude risks, suggestions for modular implementation, and expected changes.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n", 'result': ' \n\n**Impact Report: Adding Logging

In [50]:
feature_nodes_response_db
feature_edges_response_db
impact_report_response_db

'**Impact Report: Adding Logging Functionality to Major Modules**\n\n**Introduction**\n\nThe proposed feature request aims to integrate logging functionality into all major modules within the project. This report assesses the potential impact of this request on the project, highlighting risks, suggestions for modular implementation, and expected changes.\n\n**Project Overview**\n\nThe project, named "project_old," is a Flask-based application with a simple structure consisting of:\n\n1. `app.py`: The main application file.\n2. `requirements.txt`: A file listing Python dependencies, currently only including Flask.\n3. `utils` directory: Containing a single file, `helpers.py`, which provides utility functions.\n\n**Impact Analysis**\n\n**Benefits**\n\n1. **Improved Debugging**: Logging functionality will enable developers to track and diagnose issues more efficiently, reducing debugging time and effort.\n2. **Enhanced Monitoring**: Logs will provide valuable insights into system behavior

In [51]:
#analysis_results=feature_nodes_response_db+feature_nodes_response_db+impact_report_response_db

In [52]:
analysis_results=f"Nodes:\n{feature_nodes_response_db}\n\nEdges:\n{feature_edges_response_db}\n\nReport:\n{impact_report_response_db}"

In [53]:
print(analysis_results)

Nodes:
Based on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:

1. app.py (File) - Role: Main application file, likely to be a major module requiring logging functionality.
2. helpers.py (File) - Role: Utility file, possibly containing functions or classes used across the project, and thus a major module requiring logging functionality.

The requirements.txt file is not directly impacted as it only lists dependencies and does not contain code that would require logging functionality. The 'utils' directory is also not directly impacted, but its child file 'helpers.py' is.

Note that the scope of "all major modules" might be subjective, and additional analysis or clarification might be needed to determine if other files or components in the project should be considered major modules requiring logging functionality.

Edges:
Based on the project context, the feature request to add logging functionality to

In [54]:
feature_nodes

"Analyze the project context and identify the nodes (files or components) impacted by the following feature request:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nProject Context: requirements.txt: ['# Python dependencies\\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}\n\nProvide a list of impacted nodes and their roles in the project."

In [55]:
# Step 1.5: Preprocessing
logger.info("Preprocessing results of feature analysis")
preprocessing_template = get_prompt_template("preprocessing_step")
preprocessing_template
# Include the results of feature analysis (nodes and edges)
preprocessing_prompt = preprocessing_template.format(
    feature_request=user_request,
    analysis_results=f"Nodes:\n{feature_nodes_response_db}\n\nEdges:\n{feature_edges_response_db}\n\nReport:\n{impact_report_response_db}"
)
print(preprocessing_prompt)
preprocessing_response_db = query_llm(user_input=preprocessing_prompt,vector_db=vector_db)
#preprocessing_response_db

Using the results of the feature analysis, identify the specific files to be handled for the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Feature Analysis Results:
Nodes:
Based on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:

1. app.py (File) - Role: Main application file, likely to be a major module requiring logging functionality.
2. helpers.py (File) - Role: Utility file, possibly containing functions or classes used across the project, and thus a major module requiring logging functionality.

The requirements.txt file is not directly impacted as it only lists dependencies and does not contain code that would require logging functionality. The 'utils' directory is also not directly impacted, but its child file 'helpers.py' is.

Note that the scope of "all major modules" might be subjective, and additional analysis or clarification mig

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying using vector database...
Debug: Raw response from vector DB: {'query': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nNone<|eot_id|><|start_header_id|>user<|end_header_id|>\nUsing the results of the feature analysis, identify the specific files to be handled for the following feature request:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nFeature Analysis Results:\nNodes:\nBased on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:\n\n1. app.py (File) - Role: Main application file, likely to be a major module requiring logging functionality.\n2. helpers.py (File) - Role: Utility file, possibly containing functions or classes used across the project, and thus a major module requiring logging functionality.\n\nThe requirements.txt file is not directly impacted as it only lists dependencies and does not contain code that would require loggi

In [56]:
#import importlib
#import src.models.prompt_templates

#importlib.reload(src.models.prompt_templates)

In [57]:
# Step 1.6: Preprocessing request
logger.info("Preprocessing results request of feature analysis")
preprocessing_template = get_prompt_template("preprocessing_request")
preprocessing_template

'Using the results of the feature analysis, identify the specific files to be handled for the following feature request:\n\nFeature Request: {feature_request}\n\nFeature Analysis Results:\n{analysis_results}\n\nFor each impacted file:\n- Provide the file path.\n- Specify the task to be performed on the file (e.g., modify, add functionality).\n\nFor any new file required:\n- Specify the file path.\n- Provide a brief description of its purpose.\n- Suggest the initial content or structure the file should include.\n\nEnsure the output clearly separates existing files and new files to be created. Output this information as a JSON file with the following format:\n\n```json\n{\n  "feature_request": "Describe the feature concisely",\n  "analysis_results": "Provide context and analysis of the current system",\n  "existing_files": [\n    {\n      "file_path": "path/to/file.extension",\n      "task": "Modification needed in this file"\n    }\n  ],\n  "new_files": [\n    {\n      "file_path": "pat

In [58]:
preprocessing_template

'Using the results of the feature analysis, identify the specific files to be handled for the following feature request:\n\nFeature Request: {feature_request}\n\nFeature Analysis Results:\n{analysis_results}\n\nFor each impacted file:\n- Provide the file path.\n- Specify the task to be performed on the file (e.g., modify, add functionality).\n\nFor any new file required:\n- Specify the file path.\n- Provide a brief description of its purpose.\n- Suggest the initial content or structure the file should include.\n\nEnsure the output clearly separates existing files and new files to be created. Output this information as a JSON file with the following format:\n\n```json\n{\n  "feature_request": "Describe the feature concisely",\n  "analysis_results": "Provide context and analysis of the current system",\n  "existing_files": [\n    {\n      "file_path": "path/to/file.extension",\n      "task": "Modification needed in this file"\n    }\n  ],\n  "new_files": [\n    {\n      "file_path": "pat

In [59]:
feature_request

'Add logging functionality to all major modules in the project'

In [60]:
user_request 

'Add logging functionality to all major modules in the project'

In [61]:
feature_nodes_response_db 

'Based on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:\n\n1. app.py (File) - Role: Main application file, likely to be a major module requiring logging functionality.\n2. helpers.py (File) - Role: Utility file, possibly containing functions or classes used across the project, and thus a major module requiring logging functionality.\n\nThe requirements.txt file is not directly impacted as it only lists dependencies and does not contain code that would require logging functionality. The \'utils\' directory is also not directly impacted, but its child file \'helpers.py\' is.\n\nNote that the scope of "all major modules" might be subjective, and additional analysis or clarification might be needed to determine if other files or components in the project should be considered major modules requiring logging functionality.'

In [62]:
impact_report_response_db 

'**Impact Report: Adding Logging Functionality to Major Modules**\n\n**Introduction**\n\nThe proposed feature request aims to integrate logging functionality into all major modules within the project. This report assesses the potential impact of this request on the project, highlighting risks, suggestions for modular implementation, and expected changes.\n\n**Project Overview**\n\nThe project, named "project_old," is a Flask-based application with a simple structure consisting of:\n\n1. `app.py`: The main application file.\n2. `requirements.txt`: A file listing Python dependencies, currently only including Flask.\n3. `utils` directory: Containing a single file, `helpers.py`, which provides utility functions.\n\n**Impact Analysis**\n\n**Benefits**\n\n1. **Improved Debugging**: Logging functionality will enable developers to track and diagnose issues more efficiently, reducing debugging time and effort.\n2. **Enhanced Monitoring**: Logs will provide valuable insights into system behavior

```python
import logging
# Initialize the logger
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
def get_prompt_request(stage: str):
    """
    Returns a template for prompts based on the stage of the FACTACON algorithm.

    Args:
        stage (str): The stage of the algorithm.

    Returns:
        str: Prompt template for the specified stage.
    """
    prompts = {
        "preprocessing_request": (
            "Using the results of the feature analysis, identify the specific files to be handled for the following feature request:\n\n"
            "Feature Request: {feature_request}\n\n"
            "Feature Analysis Results:\n{analysis_results}\n\n"
            "For each impacted file:\n"
            "- Provide the file path.\n"
            "- Specify the task to be performed on the file (e.g., modify, add functionality).\n\n"
            "For any new file required:\n"
            "- Specify the file path.\n"
            "- Provide a brief description of its purpose.\n"
            "- Suggest the initial content or structure the file should include.\n\n"
            "Ensure the output clearly separates existing files and new files to be created. "
            "Output this information as a JSON file with the following format:\n\n"
            "```json\n"
            "{{\n"
            "  \"feature_request\": \"{feature_request}\",\n"
            "  \"analysis_results\": \" \",\n"


            "  \"existing_files\": [\n"
            "    {{\n"
            "      \"file_path\": \"path/to/file.extension\",\n"
            "      \"task\": \"Modification needed in this file\"\n"
            "    }}\n"
            "  ],\n"
            "  \"new_files\": [\n"
            "    {{\n"
            "      \"file_path\": \"path/to/new/file.extension\",\n"
            "      \"purpose\": \"Purpose of the new file\" \n"
            "    }}\n"
            "  ]\n"
            "}}\n"
            "```"
        )
    }
    return prompts.get(stage, "Stage not found.")
```

In [63]:

# Example inputs
#user_request = "Add logging functionality to all major modules in the project"

#feature_nodes_response_db = (
#    "Based on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:\n\n"
#    "1. `app.py` (File) - Role: Main application file, needs logging functionality to log important events or errors.\n\n"
#    "2. `helpers.py` (File) - Role: Utility file, needs logging functionality to log important events or errors related to helper functions.\n\n"
#    "3. `requirements.txt` (File) - Role: Dependency management file, may need to add a logging library (e.g., `logging` or a third-party library like `loguru`) as a dependency.\n\n"
#    "Note that the `utils` directory is not directly impacted, but its child file `helpers.py` is. The `flask` dependency in `requirements.txt` is not directly related to logging functionality, so it's not impacted by this feature request."
#)
#impact_report_response_db = (
#    "Impact report content describing potential changes, risks, and implementation strategies."
#)


In [64]:
from src.generation.preprocessing import get_prompt_request

In [65]:

# Logging the process
logger.info("Preprocessing results request of feature analysis")

# Get the preprocessing template
preprocessing_template = get_prompt_request("preprocessing_request")

# Generate the prompt by formatting the template with the required inputs
try:
    preprocessing_prompt = preprocessing_template.format(
        feature_request=user_request,
        analysis_results=(
            f"Nodes:\n{feature_nodes_response_db}\n\n"
            f"Edges:\n{feature_edges_response_db}\n\n"
            f"Report:\n{impact_report_response_db}"
        )
    )
    print(preprocessing_prompt)  # Display the formatted prompt for debugging or further use
except KeyError as e:
    logger.error(f"Missing key in format: {e}")
    raise


INFO:factory_feature:Preprocessing results request of feature analysis


Using the results of the feature analysis, identify the specific files to be handled for the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Feature Analysis Results:
Nodes:
Based on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:

1. app.py (File) - Role: Main application file, likely to be a major module requiring logging functionality.
2. helpers.py (File) - Role: Utility file, possibly containing functions or classes used across the project, and thus a major module requiring logging functionality.

The requirements.txt file is not directly impacted as it only lists dependencies and does not contain code that would require logging functionality. The 'utils' directory is also not directly impacted, but its child file 'helpers.py' is.

Note that the scope of "all major modules" might be subjective, and additional analysis or clarification mig

In [66]:
preprocessing_response_request_db = query_llm(user_input=preprocessing_prompt,vector_db=vector_db)
#preprocessing_response_db

Debug: Received prompt: Using the results of the feature analysis, identify the specific files to be handled for the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Feature Analysis Results:
Nodes:
Based on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:

1. app.py (File) - Role: Main application file, likely to be a major module requiring logging functionality.
2. helpers.py (File) - Role: Utility file, possibly containing functions or classes used across the project, and thus a major module requiring logging functionality.

The requirements.txt file is not directly impacted as it only lists dependencies and does not contain code that would require logging functionality. The 'utils' directory is also not directly impacted, but its child file 'helpers.py' is.

Note that the scope of "all major modules" might be subjective, and additional analy

INFO:ibm_watsonx_ai.client:Client successfully initialized
INFO:ibm_watsonx_ai.wml_resource:Successfully finished Get available foundation models for url: 'https://eu-gb.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-11-04&project_id=ac4d11fa-7292-4569-a901-f6f90c5fec86&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'
/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying using vector database...


INFO:httpx:HTTP Request: POST https://eu-gb.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-11-04 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-gb.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-11-04'


Debug: Raw response from vector DB: {'query': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nNone<|eot_id|><|start_header_id|>user<|end_header_id|>\nUsing the results of the feature analysis, identify the specific files to be handled for the following feature request:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nFeature Analysis Results:\nNodes:\nBased on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:\n\n1. app.py (File) - Role: Main application file, likely to be a major module requiring logging functionality.\n2. helpers.py (File) - Role: Utility file, possibly containing functions or classes used across the project, and thus a major module requiring logging functionality.\n\nThe requirements.txt file is not directly impacted as it only lists dependencies and does not contain code that would require logging functionality. The \'utils\' directory

In [67]:
preprocessing_response_request_db

'Here is the JSON output as requested:\n\n```json\n{\n  "feature_request": "Add logging functionality to all major modules in the project",\n  "analysis_results": "Nodes: app.py, helpers.py; Edges: app.py, helpers.py, flask",\n  "existing_files": [\n    {\n      "file_path": "project_old/app.py",\n      "task": "Modify to add logging functionality"\n    },\n    {\n      "file_path": "project_old/utils/helpers.py",\n      "task": "Modify to add logging functionality"\n    }\n  ],\n  "new_files": [\n    {\n      "file_path": "project_old/logging.py",\n      "purpose": "Centralized logging mechanism"\n    },\n    {\n      "file_path": "project_old/logging.conf",\n      "purpose": "Logging configuration file"\n    }\n  ]\n}\n```\n\nThis JSON output summarizes the feature request, analysis results, and the specific files to be handled. The "existing_files" section lists the files that need to be modified to add logging functionality, while the "new_files" section lists the new files to be c

In [68]:
import json
import re

def extract_json_from_response(response_text):
  """
  Extracts the JSON file from the given LLM response.

  Args:
    response_text: The LLM response containing the JSON file.

  Returns:
    A Python dictionary representing the JSON object, or None if no JSON is found.
  """
  try:
    # Find the JSON string using regular expression
    match = re.search(r'```json\n(.*?)```', response_text, re.DOTALL)
    if match:
      json_string = match.group(1)
      # Parse the JSON string
      json_object = json.loads(json_string)
      return json_object
    else:
      return None
  except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    return None

In [69]:
#from src.generation.preprocessing import extract_json_from_response

# Example usage

json_object = extract_json_from_response(preprocessing_response_request_db)

if json_object:
  print(json.dumps(json_object, indent=2))
else:
  print("No JSON found in the response.")

{
  "feature_request": "Add logging functionality to all major modules in the project",
  "analysis_results": "Nodes: app.py, helpers.py; Edges: app.py, helpers.py, flask",
  "existing_files": [
    {
      "file_path": "project_old/app.py",
      "task": "Modify to add logging functionality"
    },
    {
      "file_path": "project_old/utils/helpers.py",
      "task": "Modify to add logging functionality"
    }
  ],
  "new_files": [
    {
      "file_path": "project_old/logging.py",
      "purpose": "Centralized logging mechanism"
    },
    {
      "file_path": "project_old/logging.conf",
      "purpose": "Logging configuration file"
    }
  ]
}


In [70]:
print(json_object)

{'feature_request': 'Add logging functionality to all major modules in the project', 'analysis_results': 'Nodes: app.py, helpers.py; Edges: app.py, helpers.py, flask', 'existing_files': [{'file_path': 'project_old/app.py', 'task': 'Modify to add logging functionality'}, {'file_path': 'project_old/utils/helpers.py', 'task': 'Modify to add logging functionality'}], 'new_files': [{'file_path': 'project_old/logging.py', 'purpose': 'Centralized logging mechanism'}, {'file_path': 'project_old/logging.conf', 'purpose': 'Logging configuration file'}]}


In [71]:
test=False
if test:
    from src.vector_database.db_load import load_vector_db_by_collection
    from src.models.llm_inference import query_llm
    # Define the persistent directory and collection name
    persist_directory = "./notebooks/chroma_db"
    collection_name = "my_vector_collection"
    # Load the vector database
    vector_db = load_vector_db_by_collection(collection_name, persist_directory)
    # Query the vector database using the `query_llm` function
    response = query_llm(user_input=prompt, vector_db=vector_db)
    # Print the response
    print("Generated Response:")
    print(response)

In [72]:
from src.utils.tools import extract_file_content

In [73]:

# Extend the JSON object
for file_info in json_object['existing_files']:
    file_path = file_info.get('file_path', '')
    if file_path:
        file_info['content'] = extract_file_content(file_path)

# Display the updated JSON object

In [74]:

# Extend the JSON object
for file_info in json_object['existing_files']:
    file_path = file_info.get('file_path', '')
    if file_path:
        file_info['content'] = extract_file_content(file_path)

# Display the updated JSON object

In [75]:
# Display the updated JSON object
import json
print(json.dumps(json_object, indent=4))

{
    "feature_request": "Add logging functionality to all major modules in the project",
    "analysis_results": "Nodes: app.py, helpers.py; Edges: app.py, helpers.py, flask",
    "existing_files": [
        {
            "file_path": "project_old/app.py",
            "task": "Modify to add logging functionality",
            "content": "from utils.helpers import greet\n\ndef main():\n    name = input(\"Enter your name: \")\n    print(greet(name))\n\nif __name__ == \"__main__\":\n    main()"
        },
        {
            "file_path": "project_old/utils/helpers.py",
            "task": "Modify to add logging functionality",
            "content": "def greet(name):\n    \"\"\"\n    Returns a greeting message for the provided name.\n\n    Args:\n        name: The name of the person to greet.\n\n    Returns:\n        A greeting string.\n    \"\"\"\n    return f\"Hello, {name}! Welcome to the project.\""
        }
    ],
    "new_files": [
        {
            "file_path": "project_old

In [76]:
from src.utils.tools import count_tasks_from_json


In [77]:
json_data=json.dumps(json_object, indent=4)

In [78]:
# Count tasks
task_count = count_tasks_from_json(json_data)
print(f"Total tasks to perform: {task_count}")

Total tasks to perform: 4


```python
def generate_task_prompts(json_data: str) -> list:
    """
    Generates prompts for each task based on the provided JSON data.

    Args:
        json_data (str): JSON data as a string.

    Returns:
        list: A list of prompts, one for each task.
    """
    data = json.loads(json_data)
    feature_request = data["feature_request"]
    analysis_results = data["analysis_results"]
    prompts = []

    # Generate prompts for existing files
    for file in data.get("existing_files", []):
        task_prompt = (
            f"You are an expert at writing source code. Perform the following task:\n\n"
            f"Task Request: {file['task']}\n\n"
            f"Feature Request: {feature_request}\n\n"
            f"File Path: {file['file_path']}\n\n"
            f"File Content:\n{file['content']}\n\n"
            f"Analysis Results: {analysis_results}\n\n"
            f"Output ONLY the complete modified source code for the specified file. Do not include comments, explanations, or any other text."
        )
        prompts.append(task_prompt)

    # Generate prompts for new files
    for file in data.get("new_files", []):
        task_prompt = (
            f"You are an expert at writing source code. Perform the following task:\n\n"
            f"Task Request: Create a new file for {file['purpose']}\n\n"
            f"Feature Request: {feature_request}\n\n"
            f"File Path: {file['file_path']}\n\n"
            f"Analysis Results: {analysis_results}\n\n"
            f"Output ONLY the complete source code for the new file. Do not include comments, explanations, or any other text."
        )
        prompts.append(task_prompt)

    return prompts
```

In [79]:
from src.generation.task_prompts import generate_task_prompts

In [80]:
# Generate task prompts
task_responses = []
task_prompts = generate_task_prompts(json_data)




In [83]:
task_prompts

['You are an expert at writing source code. Perform the following task:\n\nTask Request: Modify to add logging functionality\n\nFeature Request: Add logging functionality to all major modules in the project\n\nFile Path: project_old/app.py\n\nFile Content:\nfrom utils.helpers import greet\n\ndef main():\n    name = input("Enter your name: ")\n    print(greet(name))\n\nif __name__ == "__main__":\n    main()\n\nAnalysis Results: Nodes: app.py, helpers.py; Edges: app.py, helpers.py, flask\n\nOutput ONLY the complete modified source code for the specified file. Do not include comments, explanations, or any other text.',
 'You are an expert at writing source code. Perform the following task:\n\nTask Request: Modify to add logging functionality\n\nFeature Request: Add logging functionality to all major modules in the project\n\nFile Path: project_old/utils/helpers.py\n\nFile Content:\ndef greet(name):\n    """\n    Returns a greeting message for the provided name.\n\n    Args:\n        name:

In [84]:
task_responses=[]
# Query the LLM for each prompt and ensure the response is limited to source code
for i, task_prompt in enumerate(task_prompts, start=1):
    print(f"\nPrompt {i}:\n{task_prompt}\n")
    task_response = query_llm(user_input=task_prompt)  # Replace with the appropriate LLM query function
    task_responses.append(task_response)

# Print the raw source code responses
for i, response in enumerate(task_responses, start=1):
    print(f"\nSource Code for Task {i}:\n{response}")
  


Prompt 1:
You are an expert at writing source code. Perform the following task:

Task Request: Modify to add logging functionality

Feature Request: Add logging functionality to all major modules in the project

File Path: project_old/app.py

File Content:
from utils.helpers import greet

def main():
    name = input("Enter your name: ")
    print(greet(name))

if __name__ == "__main__":
    main()

Analysis Results: Nodes: app.py, helpers.py; Edges: app.py, helpers.py, flask

Output ONLY the complete modified source code for the specified file. Do not include comments, explanations, or any other text.

Debug: Received prompt: You are an expert at writing source code. Perform the following task:

Task Request: Modify to add logging functionality

Feature Request: Add logging functionality to all major modules in the project

File Path: project_old/app.py

File Content:
from utils.helpers import greet

def main():
    name = input("Enter your name: ")
    print(greet(name))

if __name_

INFO:ibm_watsonx_ai.client:Client successfully initialized
INFO:ibm_watsonx_ai.wml_resource:Successfully finished Get available foundation models for url: 'https://eu-gb.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-11-04&project_id=ac4d11fa-7292-4569-a901-f6f90c5fec86&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'
/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying without vector database...


INFO:httpx:HTTP Request: POST https://eu-gb.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-11-04 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-gb.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-11-04'


Debug: Raw response from model: import logging
from utils.helpers import greet

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def main():
    logger.info("Application started")
    name = input("Enter your name: ")
    logger.info("User input received")
    logger.debug("User input: %s", name)
    result = greet(name)
    logger.info("Greeting generated")
    logger.debug("Greeting: %s", result)
    print(result)
    logger.info("Application finished")

if __name__ == "__main__":
    main()
Debug: Response as string: import logging
from utils.helpers import greet

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def main():
    logger.info("Application started")
    name = input("Enter your name: ")
    logger.info("User input received")
    logger.debug("User input: %s", name)
    result = greet(name)

...

Debug: Raw response from model: ```
import logging
import os

logging.basicConfig(filename='project.log', level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

def get_logger(name):
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    return logger

def log_app(message):
    logger = get_logger('app')
    logger.info(message)

def log_helpers(message):
    logger = get_logger('helpers')
    logger.info(message)

def log_flask(message):
    logger = get_logger('flask')
    logger.info(message)
```
Debug: Response as string: ```
import logging
import os

logging.basicConfig(filename='project.log', level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

def get_logger(name):
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    return logger

def log_app(message):
    logger = get_logger('app')
    logger.info(message)

def log_helpers(message):
    logger = get_logger('helpers')
    logge

INFO:ibm_watsonx_ai.client:Client successfully initialized
INFO:ibm_watsonx_ai.wml_resource:Successfully finished Get available foundation models for url: 'https://eu-gb.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-11-04&project_id=ac4d11fa-7292-4569-a901-f6f90c5fec86&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'
/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying without vector database...


INFO:httpx:HTTP Request: POST https://eu-gb.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-11-04 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-gb.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-11-04'


Debug: Raw response from model: ```
[loggers]
keys=root,app,helpers

[handlers]
keys=consoleHandler,fileHandler

[formatters]
keys=simpleFormatter

[logger_root]
level=DEBUG
handlers=consoleHandler,fileHandler

[logger_app]
level=DEBUG
handlers=consoleHandler,fileHandler
qualname=app
propagate=0

[logger_helpers]
level=DEBUG
handlers=consoleHandler,fileHandler
qualname=helpers
propagate=0

[handler_consoleHandler]
class=StreamHandler
level=DEBUG
formatter=simpleFormatter
args=(sys.stdout,)

[handler_fileHandler]
class=FileHandler
level=DEBUG
formatter=simpleFormatter
args=('project_old.log', 'a')

[formatter_simpleFormatter]
format=%(asctime)s - %(name)s - %(levelname)s - %(message)s
datefmt=
```
Debug: Response as string: ```
[loggers]
keys=root,app,helpers

[handlers]
keys=consoleHandler,fileHandler

[formatters]
keys=simpleFormatter

[logger_root]
level=DEBUG
handlers=consoleHandler,fileHandler

[logger_app]
level=DEBUG
handlers=consoleHandler,fileHandler
qualname=app
propagate=0

[

In [85]:
print(json_data)

{
    "feature_request": "Add logging functionality to all major modules in the project",
    "analysis_results": "Nodes: app.py, helpers.py; Edges: app.py, helpers.py, flask",
    "existing_files": [
        {
            "file_path": "project_old/app.py",
            "task": "Modify to add logging functionality",
            "content": "from utils.helpers import greet\n\ndef main():\n    name = input(\"Enter your name: \")\n    print(greet(name))\n\nif __name__ == \"__main__\":\n    main()"
        },
        {
            "file_path": "project_old/utils/helpers.py",
            "task": "Modify to add logging functionality",
            "content": "def greet(name):\n    \"\"\"\n    Returns a greeting message for the provided name.\n\n    Args:\n        name: The name of the person to greet.\n\n    Returns:\n        A greeting string.\n    \"\"\"\n    return f\"Hello, {name}! Welcome to the project.\""
        }
    ],
    "new_files": [
        {
            "file_path": "project_old

In [86]:
task_responses

['import logging\nfrom utils.helpers import greet\n\nlogging.basicConfig(level=logging.INFO, format=\'%(asctime)s - %(name)s - %(levelname)s - %(message)s\')\nlogger = logging.getLogger(__name__)\n\ndef main():\n    logger.info("Application started")\n    name = input("Enter your name: ")\n    logger.info("User input received")\n    logger.debug("User input: %s", name)\n    result = greet(name)\n    logger.info("Greeting generated")\n    logger.debug("Greeting: %s", result)\n    print(result)\n    logger.info("Application finished")\n\nif __name__ == "__main__":\n    main()',
 'import logging\n\nlogging.basicConfig(level=logging.INFO)\nlogger = logging.getLogger(__name__)\n\ndef greet(name):\n    logger.info("Generating greeting message for %s", name)\n    return f"Hello, {name}! Welcome to the project."\n    logger.info("Greeting message generated successfully")',
 "```\nimport logging\nimport os\n\nlogging.basicConfig(filename='project.log', level=logging.INFO, format='%(asctime)s - 

```
import os
import shutil
import logging
import json

# Initialize the logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def update_project_structure(json_data: str, task_responses: list, old_project_path: str, new_project_path: str, overwrite: bool = True):
    """
    Updates the project structure by cloning the original project, modifying files based on task responses,
    and saving the updated files into a new project directory.

    Args:
        json_data (str): JSON data as a string.
        task_responses (list): List of source code responses for each task.
        old_project_path (str): Path to the original project directory.
        new_project_path (str): Path to save the new project directory.
        overwrite (bool): Whether to overwrite existing files in the new project structure.
    """
    # Step 1: Parse the JSON data
    data = json.loads(json_data)
    existing_files = data.get("existing_files", [])

    # Step 2: Clone the original project structure
    logger.info("Cloning the original project structure")
    if os.path.exists(new_project_path):
        shutil.rmtree(new_project_path)
    shutil.copytree(old_project_path, new_project_path)

    # Step 3: Update modified files with task responses
    logger.info("Updating modified files based on task responses")
    for i, file_info in enumerate(existing_files):
        old_file_path = file_info["file_path"]
        new_file_path = old_file_path.replace("project_old/", "project_new/")
        updated_content = task_responses[i]

        # Check if file exists and handle overwrite flag
        if not overwrite and os.path.exists(new_file_path):
            logger.info(f"File exists and overwrite is disabled: {new_file_path}. Keeping the original file.")
            continue

        # Ensure the new file directory exists
        new_file_dir = os.path.dirname(new_file_path)
        if not os.path.exists(new_file_dir):
            os.makedirs(new_file_dir)

        # Write the updated content to the new file
        with open(new_file_path, "w") as f:
            f.write(updated_content)
        logger.info(f"Updated file saved: {new_file_path}")

    # Step 4: Verify the new project structure
    logger.info("New project structure generated successfully")
    for root, _, files in os.walk(new_project_path):
        for file in files:
            logger.info(f"File in new project: {os.path.join(root, file)}")


```

In [87]:
from src.generation.project_structure import update_project_structure

In [88]:
# Define project paths
old_project_path = "project_old"
new_project_path = "project_new"

# Run the update pipeline with overwrite flag (default: True)
update_project_structure(json_data, task_responses, old_project_path, new_project_path, overwrite=True)

# To disable overwriting, call as follows:
# update_project_structure(json_data, task_responses, old_project_path, new_project_path, overwrite=False)


INFO:src.generation.project_structure:Cloning the original project structure
INFO:src.generation.project_structure:Updating modified files based on task responses
INFO:src.generation.project_structure:Updated file saved: project_new/app.py
INFO:src.generation.project_structure:Updated file saved: project_new/utils/helpers.py
INFO:src.generation.project_structure:New project structure generated successfully
INFO:src.generation.project_structure:File in new project: project_new/app.py
INFO:src.generation.project_structure:File in new project: project_new/requirements.txt
INFO:src.generation.project_structure:File in new project: project_new/utils/helpers.py


```
import os
import json
import logging

# Initialize the logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_expected_files_from_json(json_data: str) -> list:
    """
    Extracts the expected file paths from the given JSON data.

    Args:
        json_data (str): JSON data as a string.

    Returns:
        list: A list of expected file paths relative to the project root.
    """
    data = json.loads(json_data)
    existing_files = data.get("existing_files", [])
    expected_files = [file["file_path"].replace("project_old/", "") for file in existing_files]
    return expected_files


def validate_project_consistency(new_project_path: str, expected_files: list):
    """
    Validates the consistency of the new project structure by checking for expected files.

    Args:
        new_project_path (str): Path to the new project directory.
        expected_files (list): List of expected file paths relative to the project root.

    Returns:
        bool: True if all expected files are present, False otherwise.
    """
    logger.info("Validating consistency of the project tree")
    missing_files = []

    for expected_file in expected_files:
        expected_path = os.path.join(new_project_path, expected_file)
        if not os.path.exists(expected_path):
            missing_files.append(expected_path)

    if missing_files:
        logger.error("Validation failed. Missing files:")
        for file in missing_files:
            logger.error(file)
        return False

    logger.info("Validation passed. All files are present in the new project structure.")
    return True

```

In [89]:
from src.generation.project_structure import create_expected_files_from_json, validate_project_consistency

In [90]:
# Create expected files list from JSON
expected_files = create_expected_files_from_json(json_data)

# Run the validation function
new_project_path = "project_new"
is_valid = validate_project_consistency(new_project_path, expected_files)

if is_valid:
    logger.info("The project structure is consistent.")
else:
    logger.error("The project structure is inconsistent.")


INFO:src.generation.project_structure:Validating consistency of the project tree
INFO:src.generation.project_structure:Validation passed. All files are present in the new project structure.
INFO:factory_feature:The project structure is consistent.


In [91]:
logger.info("Feature integration completed")
print(f"Project updated with the feature: {user_request}")



INFO:factory_feature:Feature integration completed


Project updated with the feature: Add logging functionality to all major modules in the project


In [92]:

logger.info("Feature integration completed")
print(f"Project updated with the feature: {prompt}")
print(f"Updated project saved in: {new_project_path}")


INFO:factory_feature:Feature integration completed


Project updated with the feature: Who is man's best friend?
Updated project saved in: project_new


```
def main(prompt):
    logger.info("Starting the Factory Feature program")
    # Define paths
    old_project_path = "project_old"
    new_project_path = "project_new"

    # Step 1: Parse the old project
    logger.info("Parsing the project structure")

    logger.info("Vector database created for the project")

    # Step 2: Resolve dependencies
    logger.info("Resolving project dependencies")
  
    logger.info(f"Dependencies found: {dependencies.keys()}")

    # Step 3: Map features to project components
    logger.info("Mapping feature request to project components")
    logger.info(f"Relevant documents for the feature request: {[doc.metadata['source'] for doc in relevant_docs]}")

    # Step 4: Generate new project structure
    logger.info("Generating the updated project structure")

    # Step 5: Integrate new features
    logger.info("Integrating new features into the project")

    # Step 6: Generate additional files or modify existing ones


    logger.info("Feature integration completed")
    print(f"Project updated with the feature: {prompt}")
    print(f"Updated project saved in: {new_project_path}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Factory Feature Program")
    parser.add_argument("--prompt", required=True, help="Feature request prompt")
    args = parser.parse_args()

    try:
        main(args.prompt)
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        raise

```